# 教師あり二値分類（奇数が正）のノートブック

In [1]:
from matplotlib import pyplot as plt
import polars as pl
import seaborn as sns
import japanize_matplotlib

from common import *

In [2]:
# ハイパーパラメータ
N_COMPONENTS = 50
TEST_SIZE = 0.2
VAL_SIZE = 0.2

In [3]:
# データセットの読み込み
X_train, X_val, X_test, y_train, y_val, y_test = load_mnist_pca_train_test_val(
    n_components=N_COMPONENTS, test_size=TEST_SIZE, val_size=VAL_SIZE
)

Files already downloaded
Files already downloaded


In [4]:
# ラベルの書き換え（奇数が正）
X_train, y_train = rewrite_label_with_binary_setting(X_train, y_train)
X_val, y_val = rewrite_label_with_binary_setting(X_val, y_val)
X_test, y_test = rewrite_label_with_binary_setting(X_test, y_test)

In [5]:
def train_lgbm_with_custom_loss(X_train, X_valid, y_train, y_valid):

    """Train LightGBM with custom loss function."""
    train_data = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
    valid_data = lgb.Dataset(X_valid, label=y_valid, free_raw_data=False)

    params = {
        "objective": "binary",
        "verbose": -1,
        "learning_rate": 0.01,
        "num_boost_round": 400,
    }
    valid_accuracies = []

    def record_accuracies(p: lgb.Booster, train_data: lgb.Dataset, valid_data: lgb.Dataset):
        valid_pred = (p.predict(valid_data.data) > 0.5).astype(int)
        valid_acc = accuracy_score(valid_data.label, valid_pred)
        valid_accuracies.append(valid_acc)
        print(valid_acc)
    
    gbm = lgb.train(
        params,
        train_data,
        valid_sets=[valid_data],
        callbacks=[lambda p: record_accuracies(p.model, train_data, valid_data)]
    )
    return gbm, valid_accuracies

In [6]:
gbm, valid_accuracies =train_lgbm_with_custom_loss(X_train, X_val, y_train, y_val)
y_pred = gbm.predict(X_val)
print("Accuracy: {}".format(accuracy_score(y_val, (y_pred > 0.5).astype(int))))

/Users/ryoma.kobayashi/gits/techblog_git/wsl_w_lgbm/.venv/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


0.5055
0.5574166666666667
0.79525
0.82325
0.8445833333333334
0.85025
0.8541666666666666
0.85675
0.8559166666666667
0.85775
0.8598333333333333
0.86275
0.8625
0.86125
0.8625
0.8639166666666667
0.86425
0.8655
0.8663333333333333
0.8676666666666667
0.869
0.869
0.8698333333333333
0.87275
0.8733333333333333
0.8740833333333333
0.8749166666666667
0.8756666666666667
0.8763333333333333
0.87675
0.8770833333333333
0.87825
0.8793333333333333
0.8805833333333334
0.881
0.8816666666666667
0.88325
0.8829166666666667
0.8835833333333334
0.8835
0.884
0.8844166666666666
0.8845833333333334
0.8853333333333333
0.8859166666666667
0.8868333333333334
0.8866666666666667
0.8871666666666667
0.8870833333333333
0.88775
0.88825
0.8880833333333333
0.8885
0.8890833333333333
0.88925
0.89
0.8905
0.8910833333333333
0.8909166666666667
0.8918333333333334
0.8924166666666666
0.8933333333333333
0.8925
0.8924166666666666
0.89225
0.8925833333333333
0.8925
0.8926666666666667
0.8931666666666667
0.8934166666666666
0.8933333333333333
0

In [7]:
np.save("sup_bin_valid_accuracies.npy", valid_accuracies)